In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import dask.dataframe as dd
import holoviews as hv
import datashader as ds
from holoviews.operation.datashader import datashade, dynspread

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Load train data

In [ ]:
%%time
#df_train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv', engine="pyarrow")
df_train = pd.read_parquet('/kaggle/working/train.parquet')
df_train.info()

In [ ]:
df_train = dd.from_pandas(df_train, npartitions=20)
df_train.head()

# 2. Preprocessing

## 2.1 Sampling

In [ ]:
print(f'Number of stocks: {len(df_train["stock_id"].unique())}')

In [ ]:
df_subset = df_train[(df_train['stock_id'].isin([1,2])) & (df_train['date_id'] % 5 == 0)]
print(len(df_subset))

# 3. EDA

## 3.1 Plotting against time

In [ ]:
%%time
num_rows = 4
num_cols = 4
fig, axes = plt.subplots(num_rows, num_cols, figsize=(24,16))
for i, column in enumerate(df_train.columns.tolist()[2:-2]):
    row_idx = i // num_cols  # Calculate row index for subplots
    col_idx = i % num_cols   # Calculate column index for subplots
    ax = axes[row_idx][col_idx]
    sns.lineplot(data= df_subset, x='date_id', y=column, hue='stock_id',ax=ax)
    
plt.tight_layout()
plt.show()

## 3.2 Distributions

In [ ]:
df_subset_pd = df_subset.compute()
sns.boxplot(x='stock_id', y='target', data=df_subset_pd)

# 4. Training

## 4.1 Validation Split

In [ ]:
df_val = df_train.loc[df_train['time_id'].isin(df_train['time_id'].unique()[-6000:])]
df_train = df_train.loc[df_train['time_id'].isin(df_train['time_id'].unique()[:-6000])]
df_train

In [ ]:
sns.lineplot(data= df_val[df_val['stock_id'] == 1], x='date_id', y=column, hue='stock_id')

In [ ]:
sns.lineplot(data= df_train[df_train['stock_id'] == 1], x='date_id', y=column, hue='stock_id')